In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
from tradingcore.execution_manager import ExecutionManager
from tradingcore.campaign import Campaign
from tradingcore.account import Account
from tradingcore.moneymanagement import PlainMM

from exobuilder.data.datasource_mongo import DataSourceMongo
from exobuilder.data.datasource_sql import DataSourceSQL
from exobuilder.data.assetindex_mongo import AssetIndexMongo
from exobuilder.data.exostorage import EXOStorage

import pprint

In [ ]:
try:
    from .settings import *
except SystemError:
    from scripts.settings import *

try:
    from .settings_local import *
except SystemError:
    try:
        from scripts.settings_local import *
    except ImportError:
        pass
    pass

In [ ]:
assetindex = AssetIndexMongo(MONGO_CONNSTR, MONGO_EXO_DB)
storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
#datasource = DataSourceSQL(SQL_HOST, SQL_USER, SQL_PASS, assetindex, futures_limit=10, options_limit=10)
datasource = DataSourceMongo(MONGO_CONNSTR, MONGO_EXO_DB, assetindex, futures_limit=10, options_limit=10, exostorage=storage)


In [ ]:
exmgr = ExecutionManager(MONGO_CONNSTR, datasource, dbname=MONGO_EXO_DB)

In [ ]:
campaigns = exmgr.campaign_load_all()
accounts = exmgr.account_load_all()
pp = pprint.PrettyPrinter(indent=4)

## List of available campaigns

In [ ]:
[print(x) for x in campaigns.keys()];

In [ ]:
swm_info = storage.swarms_info()
pp.pprint(swm_info)

# Alphas list

In [ ]:
instruments_filter = ['NG']  # Select ALL
#instruments_filter = ['ES', 'CL']

exo_filter = ['*'] # Select ALL
#exo_filter = ['BullishCollar']

direction_filter = [0, -1, 1] # Select ALL
#direction_filter = [1]

alpha_filter = ['']  # Select ALL
#alpha_filter = ['MaCross', 'PointAndFigurePatterns']

swmdf, swm_data = storage.swarms_list(instruments_filter, direction_filter, alpha_filter, exo_filter)

[print(s) for s in sorted(swmdf.columns)];

In [ ]:
passive_exo = {}

exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
exo_filter = 'NG*'  # ES Collars (incl Bearish, Bullish, BW and vanilla)


print('==== Passive EXO list')
for exo_name in storage.exo_list(exo_filter):
    passive_exo[exo_name], exo_dict = storage.load_series(exo_name)

for exo in sorted(passive_exo.keys()):
    print(exo)
    
print('==== Alphas EXO list')
[print(s) for s in sorted(swmdf.columns)];

In [ ]:
swmdf.plot().legend(loc='best', bbox_to_anchor=(1, 0.5));

# Campaign composition

In [ ]:
 campaign_dict = {
            'name': 'NG_Bull',
            'description': 'Bullish Expression of Natural Gas',
            'alphas': {
                'NG_CallSpread_Long_BolllingerBands_alt2-bullish-custom': {
                    'qty': 1.0,
                },
             'NG_CallSpread_Long_BolllingerBands_alt3-bullish-custom': {
                    'qty': 1.0,
                },
             'NG_CallSpread_Long_BolllingerBands_bullish-custom': {
                    'qty': 1.0,
                },
             'NG_CallSpread_Long_IchimokuCloud_bullish-custom': {
                    'qty': 1.0,
                },
             'NG_CallSpread_Long_SwingPoint_bullish-custom': {
                    'qty': 1.0,
                },
            'NG_CallSpread_Short_SwingPoint_bearish-custom': {
                    'qty': 1.0,
                },
            'NG_PutSpread_Long_IchimokuCloud_alt3-bullish-custom': {
                    'qty': 1.0,
                },
            'NG_PutSpread_Long_IchimokuCloud_bullish-custom': {
                    'qty': 1.0,
                },
            'NG_PutSpread_Short_EXO': {
                    'qty': 1.0,
                },
            
            
            }
        }

In [ ]:
cmp = Campaign(campaign_dict, datasource)

### Save campaign

In [ ]:
exmgr.campaign_save(cmp)

## Campaign positions

### Alphas positions

In [ ]:
pp.pprint(cmp.alphas_positions)

### EXO net positions

In [ ]:
pp.pprint(cmp.exo_positions)

### Contracts positions

In [ ]:
pp.pprint(cmp.positions)